# Python для сбора и анализа данных

*Алла Тамбовцева, НИУ ВШЭ*

## Практикум 4. Работа с API ВКонтакте: собираем посты со стены, комментарии к постам и информацию пользователях

## Подготовка к работе

Загружаем модули и библиотеки, необходимые для работы:

In [1]:
import requests
import time
import pandas as pd

Для начала давайте посмотрим на документацию API и посмотрим, как к нему формировать запросы: https://dev.vk.com/api/api-requests.
По [инструкции](https://allatambov.github.io/pypolit/vk-auth.pdf) мы можем получить доступ к API, создадим приложение и скопируем его ID, чтобы получить ссылку для токена доступа:

In [2]:
app_id = input("Enter your client id: ")
url = f"https://oauth.vk.com/authorize?client_id={app_id}&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token"
print(url)

Enter your client id: 51450878
https://oauth.vk.com/authorize?client_id=51450878&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token


Переходим по ссылке выше и копируем ссылку (полностью!) из адресной строки:

In [3]:
full_link = input()

https://oauth.vk.com/blank.html#access_token=vk1.a.d5lZc5knq-i5e11qs8vfbRdM0T3SRxHxAjEb6vcNs8aVfwK6LclQ3j7u4nQ2_lvTQTyurDMzBoDwA0fH_qtAoSWnaS2xRVeitUDbJOZK5IWicFjmXw-GtUh95IlkjrLim1x07CUbTt7IGtkU6WZ-FIXWmObZ4-Tw7puFjV9k8gp6_cE1BN9RY7alyU1DjJ2m&expires_in=86400&user_id=20473269


Разбиваем полученную ссылку (сначала по `access_token=`, потом часть после `=` по `&`) и забираем токен в чистом виде:

In [5]:
token = full_link.split("access_token=")[1].split("&")[0]

Ура! Теперь токен доступа у нас есть, всё готово к работе!

## Часть 1: выгружаем посты со стены сообщества

На этом практическом занятии мы будем выгружать посты из сообщества [Цитатник ВШЭ](https://vk.com/hseteachers). Сохраним в переменные версию API, ссылку для метода работы со стеной сообщества и название сообщества:

In [6]:
v = "5.131"
main_wall = "https://api.vk.com/method/wall.get"
domain = "hseteachers"

Функция `get()` из библиотеки `requests` умеет подставлять в запрос необходимые параметры и объединять их с помощью `?` и `&`. Сохраним необходимые параметры в виде словаря:

In [7]:
params_wall = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v}

А теперь сформируем запрос и выгрузим результаты в формате JSON – в Python данные в таком формате будут представлены в виде словаря:

In [8]:
req_wall = requests.get(main_wall, params = params_wall)

In [9]:
# код 200 – есть результат, нет ошибки соединения
req_wall

<Response [200]>

In [10]:
# забираем из объекта типа «запрос» данные в JSON
# раскомментируйте результат – это очень большой словарь

json_wall = req_wall.json()
#json_wall

Извлечём из этого большого словаря элемент, который отвечает за общее число постов на стене:

In [11]:
# сначала забираем значение по ключу response, 
# а потом из нового словаря забираем значение по ключу count

nposts = json_wall['response']['count']
print(nposts)

9646


Теперь извлечём элемент, который хранит результаты – список из маленьких словарей с информацией о постах (1 словарь = 1 пост):

In [12]:
items_wall = json_wall['response']['items']

In [13]:
# items_all – список со словарями
# пока там 100 постов, vk не дает за раз сгрузить все nposts постов

len(items_wall)

100

Посмотрим на один элемент такого списка:

In [14]:
i = items_wall[0]
i

{'donut': {'is_donut': False},
 'comments': {'can_post': 1, 'count': 0},
 'marked_as_ads': 0,
 'zoom_text': True,
 'short_text_rate': 0.8,
 'hash': 'iqQ6ojB-whQOlmzHJyirKqTTpZs',
 'type': 'post',
 'attachments': [],
 'date': 1679140200,
 'from_id': -63442801,
 'id': 38481,
 'is_favorite': False,
 'likes': {'can_like': 1,
  'count': 96,
  'user_likes': 0,
  'can_publish': 1,
  'repost_disabled': False},
 'owner_id': -63442801,
 'post_source': {'type': 'api'},
 'post_type': 'post',
 'reposts': {'count': 21, 'user_reposted': 0},
 'text': 'Источник познания - фигня из головы.\n\n#Виноградов_ВШЭ #Налоговое_регулирования',
 'views': {'count': 2808}}

### Задача 1

Извлеките из элемента `i` следующие компоненты:

* id поста;
* дата поста;
* текст поста;
* число лайков;
* число репостов;
* число просмотров;
* число комментариев.

In [15]:
id_ = i["id"]
date = i["date"]
text = i["text"]

# информация по лайкам и подобным – в маленьких словарях
# нужно дополнительно искать запись по ключу count

nlikes = i["likes"]["count"]
nrepos = i["reposts"]["count"]
nviews = i["views"]["count"]
ncomments = i["comments"]["count"]

print(id_, date, text)
print(nlikes, nrepos, nviews, ncomments)

38481 1679140200 Источник познания - фигня из головы.

#Виноградов_ВШЭ #Налоговое_регулирования
96 21 2808 0


### Задача 2

Изучить один пост и понять, что нам от него нужно, это хорошо, но, конечно, мы захотим выгрузить все посты сразу, а уже потом разобраться, какую информацию о них нам оставить. Ограничения данного API таковы, что за один раз мы можем выгрузить только 100 постов. Хорошие новости: каждый раз при выгрузке мы можем начинать с того поста, на котором закончили, то есть сначала выгрузить первые 100 постов, потом – следующие 100 постов, и так до тех пор, пока не заполучим все. 

Общее число постов сохранено в `nposts`. Посчитайте, сколько раз нужно будет выполнить выгрузку по 100 постов, чтобы собрать все тексты, и сохраните его в переменную `iterate`.

In [16]:
iterate = nposts // 100 + 1

### Задача 3

Прочитайте в документации к API ВКонтакте про аргумент `offset` в методе `wall.get`. Используя полученную информацию и блоки кода ниже, выгрузите и сохраните в список `items_all` данные по всем постам на стене сообщества.

**Подсказка:** чтобы расширять список правильным образом, используйте метод `.extend()`, а не `.append()`, он добавляет не один элемент, а сразу несколько.

In [17]:
params_wall_long = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v,
              "offset" : 0}

In [18]:
items_all = []

for i in range(iterate):
    req_wall_long = requests.get(main_wall, params = params_wall_long)
    json_wall_long = req_wall_long.json()
    items_wall_long = json_wall_long['response']['items']
    
    # через extend() добавляем элементы
    # обновляем параметр сдвига на каждом шаге на 100
    # ждем 1.5 секунды – выжидаем время между запросами 
    # (можно меньше, но хотя бы 1 секунды)
    
    items_all.extend(items_wall_long)
    params_wall_long["offset"] = params_wall_long["offset"] + 100
    
    time.sleep(1.5)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96


Проверяем длину списка – все ли посты собраны:

In [19]:
len(items_all)

9646

Не отбираем ничего на этом этапе, просто превращаем список словарей в датафрейм:

In [20]:
# датафрейм = таблица
# пока не обсуждали, но очень полезно:
# список списков/список кортежей/список словарей
# можно привести к удобному табличному виду 
# с помощью DataFrame() из pandas

df = pd.DataFrame(items_all)
df

,donut,comments,marked_as_ads,zoom_text,short_text_rate,hash,type,attachments,date,from_id,...,owner_id,post_source,post_type,reposts,text,views,carousel_offset,edited,copy_history,signer_id
0,{'is_donut': False},"{'can_post': 1, 'count': 0}",0,True,0.8,iqQ6ojB-whQOlmzHJyirKqTTpZs,post,[],1679140200,-63442801,...,-63442801,{'type': 'api'},post,"{'count': 21, 'user_reposted': 0}",Источник познания - фигня из головы.\n\n#Виног...,{'count': 2810},NaN,NaN,NaN,NaN
1,{'is_donut': False},"{'can_post': 1, 'count': 1}",0,True,0.8,Q79sg-IgNZu9Ozadaf65T2hbcdk,post,[],1679053800,-63442801,...,-63442801,{'type': 'api'},post,"{'count': 8, 'user_reposted': 0}",Поцелуй не существует вне двух пар губ.\n\n#Де...,{'count': 3229},NaN,NaN,NaN,NaN
2,{'is_donut': False},"{'can_post': 1, 'count': 1}",0,NaN,0.8,vhAeEOiK4OsftDYvOcFD7LNOdSQ,post,[],1678967400,-63442801,...,-63442801,{'type': 'api'},post,"{'count': 29, 'user_reposted': 0}","*Преподаватель перепутал сигму с эпсилон*\nНу,...",{'count': 4680},NaN,NaN,NaN,NaN
3,{'is_donut': False},"{'can_post': 1, 'count': 0}",0,NaN,0.8,UTrM_ilo5x-6eqbJOEtoeFKwc1s,post,[],1678881000,-63442801,...,-63442801,{'type': 'api'},post,"{'count': 5, 'user_reposted': 0}",*о плохих работах:\n\nGarbage in - garbage out...,{'count': 2890},NaN,NaN,NaN,NaN
4,{'is_donut': False},"{'can_post': 1, 'count': 6}",0,NaN,0.8,cwK0co05371aN2E-FGJhfSln6lk,post,[],1678794600,-63442801,...,-63442801,{'type': 'api'},post,"{'count': 22, 'user_reposted': 0}",Для сельского главы не важно уверенное владени...,{'count': 4004},NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9641,{'is_donut': False},"{'can_post': 1, 'count': 0}",0,NaN,0.8,R0Fe-svp1nNPFvESJvd2i7IxgQc,post,[],1387918264,-63442801,...,-63442801,{'type': 'vk'},post,"{'count': 0, 'user_reposted': 0}",по маленькой кругленькой громов\n#Громов_hse\n...,NaN,NaN,NaN,NaN,NaN
9642,{'is_donut': False},"{'can_post': 1, 'count': 2}",0,True,0.8,J0rwvUYFJwF7TGPd18CX-f7Zjh4,post,[],1387917399,-63442801,...,-63442801,{'type': 'vk'},post,"{'count': 1, 'user_reposted': 0}",Все глоки суть куздры (Данько)\n#Данько_hse,NaN,NaN,NaN,NaN,NaN
9643,{'is_donut': False},"{'can_post': 1, 'count': 0}",0,True,0.8,HrUMvlwK0CKS7qpG5jMz41la84Y,post,[],1387917345,-63442801,...,-63442801,{'type': 'vk'},post,"{'count': 2, 'user_reposted': 0}",с какого бадуна ты это написал? (Самовол)\n#Са...,NaN,NaN,NaN,NaN,NaN
9644,{'is_donut': False},"{'can_post': 1, 'count': 0}",0,True,0.8,knAD_MHjSqBG4qUNxjra7A0kkTE,post,[],1387916061,-63442801,...,-63442801,{'type': 'vk'},post,"{'count': 0, 'user_reposted': 0}",Синдром яндекса (Шаповалов И. А)\n#Шаповалов_hse,NaN,NaN,NaN,NaN,NaN


### Задача 4

Создайте на основе датафрейма `df` новый датафрейм `small` со следующими столбцами:

* id поста (`id`);
* дата поста (`date`);
* текст поста (`text`);
* число лайков (`nlikes`);
* число просмотров (`nviews`);
* число комментариев (`ncomments`).

**Подсказка:** отберите сначала все столбцы с нужной информацией, напишите функцию для извлечения только числа лайков/просмотров/комментариев и примените её, удалите лишние столбцы.

In [21]:
# выбираем столбцы по названиям
# внешние квадратные скобки – для выбора
# внутренние – границы списка, несколько названий оформляем в список

small = df[["id", "date", "text", "likes", "views", "comments"]]

In [22]:
# удалим строки с пропущенными значениями
# в старых постах не фиксировались просмотры – во views есть NaN

small = small.dropna() 

In [23]:
# первые 5 строк

small.head(5)

,id,date,text,likes,views,comments
0,38481,1679140200,Источник познания - фигня из головы.\n\n#Виног...,"{'can_like': 1, 'count': 96, 'user_likes': 0, ...",{'count': 2810},"{'can_post': 1, 'count': 0}"
1,38477,1679053800,Поцелуй не существует вне двух пар губ.\n\n#Де...,"{'can_like': 1, 'count': 71, 'user_likes': 0, ...",{'count': 3229},"{'can_post': 1, 'count': 1}"
2,38475,1678967400,"*Преподаватель перепутал сигму с эпсилон*\nНу,...","{'can_like': 1, 'count': 122, 'user_likes': 0,...",{'count': 4680},"{'can_post': 1, 'count': 1}"
3,38439,1678881000,*о плохих работах:\n\nGarbage in - garbage out...,"{'can_like': 1, 'count': 44, 'user_likes': 0, ...",{'count': 2890},"{'can_post': 1, 'count': 0}"
4,38433,1678794600,Для сельского главы не важно уверенное владени...,"{'can_like': 1, 'count': 138, 'user_likes': 0,...",{'count': 4004},"{'can_post': 1, 'count': 6}"


In [24]:
# пока не обсуждали, но не очень сложная вещь
# метод .apply() позволяет применять некоторую функцию (операцию)
# к каждой ячейке в столбце, безо всяких циклов и подобного,
# вроде map() только для столбцов и таблиц

# нам надо из каждой ячейки со словарями вида {'count': 2810}
# забрать числа, поэтому функция у нас тут такая:
# забирай значение x в ячейке (lambda x:)
# и извлекай из него запись с ключом count (x["count"])

# когда значения извлекли – сохраняем их в новые столбцы
# small["nlikes"], small["nviews"], small["ncomments"]
# похоже на добавление новых записей в словари

small["nlikes"] = small["likes"].apply(lambda x: x["count"])
small["nviews"] = small["views"].apply(lambda x: x["count"])
small["ncomments"] = small["comments"].apply(lambda x: x["count"])
small

,id,date,text,likes,views,comments,nlikes,nviews,ncomments
0,38481,1679140200,Источник познания - фигня из головы.\n\n#Виног...,"{'can_like': 1, 'count': 96, 'user_likes': 0, ...",{'count': 2810},"{'can_post': 1, 'count': 0}",96,2810,0
1,38477,1679053800,Поцелуй не существует вне двух пар губ.\n\n#Де...,"{'can_like': 1, 'count': 71, 'user_likes': 0, ...",{'count': 3229},"{'can_post': 1, 'count': 1}",71,3229,1
2,38475,1678967400,"*Преподаватель перепутал сигму с эпсилон*\nНу,...","{'can_like': 1, 'count': 122, 'user_likes': 0,...",{'count': 4680},"{'can_post': 1, 'count': 1}",122,4680,1
3,38439,1678881000,*о плохих работах:\n\nGarbage in - garbage out...,"{'can_like': 1, 'count': 44, 'user_likes': 0, ...",{'count': 2890},"{'can_post': 1, 'count': 0}",44,2890,0
4,38433,1678794600,Для сельского главы не важно уверенное владени...,"{'can_like': 1, 'count': 138, 'user_likes': 0,...",{'count': 4004},"{'can_post': 1, 'count': 6}",138,4004,6
...,...,...,...,...,...,...,...,...,...
6482,11985,1484070322,"С: А можете объяснить, чем отличается случайно...","{'can_like': 1, 'count': 101, 'user_likes': 0,...",{'count': 6198},"{'can_post': 1, 'count': 3}",101,6198,3
6483,11983,1484064169,- I think we will need more spise explorers.\n...,"{'can_like': 1, 'count': 49, 'user_likes': 0, ...",{'count': 5530},"{'can_post': 1, 'count': 1}",49,5530,1
6484,11982,1484059529,В свободное от лекций время я сижу со своим ма...,"{'can_like': 1, 'count': 134, 'user_likes': 0,...",{'count': 6006},"{'can_post': 1, 'count': 0}",134,6006,0
6485,11976,1484054073,"""Что-то вас много... Надо над этим поработать....","{'can_like': 1, 'count': 133, 'user_likes': 0,...",{'count': 6005},"{'can_post': 1, 'count': 0}",133,6005,0


In [25]:
# удаляем ненужные столбцы – числа мы уже извлекли
# метод drop() удалет строки или столбцы, здесь столбцы columns

small = small.drop(columns = ["likes", "views", "comments"])
small

,id,date,text,nlikes,nviews,ncomments
0,38481,1679140200,Источник познания - фигня из головы.\n\n#Виног...,96,2810,0
1,38477,1679053800,Поцелуй не существует вне двух пар губ.\n\n#Де...,71,3229,1
2,38475,1678967400,"*Преподаватель перепутал сигму с эпсилон*\nНу,...",122,4680,1
3,38439,1678881000,*о плохих работах:\n\nGarbage in - garbage out...,44,2890,0
4,38433,1678794600,Для сельского главы не важно уверенное владени...,138,4004,6
...,...,...,...,...,...,...
6482,11985,1484070322,"С: А можете объяснить, чем отличается случайно...",101,6198,3
6483,11983,1484064169,- I think we will need more spise explorers.\n...,49,5530,1
6484,11982,1484059529,В свободное от лекций время я сижу со своим ма...,134,6006,0
6485,11976,1484054073,"""Что-то вас много... Надо над этим поработать....",133,6005,0


### Задача 5

Выберите только те строки в полученном датафрейме, которые соответствуют постам с числом комментариев больше 0, и сохраните их в датафрейм `with_comm`.

In [26]:
# шаг 1: формулируем условие сразу для всего столбца,
# возвращается набор из True и False 
# для каждой ячейки – выполняется ли условие на ней или нет

small["ncomments"] > 0

0       False
1        True
2        True
3       False
4        True
        ...  
6482     True
6483     True
6484    False
6485    False
6486    False
Name: ncomments, Length: 6487, dtype: bool

In [27]:
# шаг 2: помещаем это условие в квадратные скобки
# смысл: отбери все строки из small, где условие в скобках возвращает True

with_comm = small[small["ncomments"] > 0] 

## Часть 2: собираем комментарии к постам

В целях экономии времени не будем брать весь датафрейм `with_comm`, для примера отберем первые 10 строк:

In [28]:
with_comm10 = with_comm.head(10) 

Заберем id постов и преобразуем их в список:

In [29]:
ids = list(with_comm10["id"])

Возьмем один id для примера и сформулируем запрос для получения комментариев к посту с этим id:

In [32]:
i = 38433

# метод wall.getComments
# owner_id: id сообщества
# post_id: id поста
# v: версия API, все та же
# access_token: токен доступа, все тот же
# count: число комментариев, берем 100, max возможное за раз
# thread_items_count: число ответов на комментарий, 10, max за раз

main_comm = "https://api.vk.com/method/wall.getComments"
params_comm = {"owner_id" : -63442801,
              "post_id" : i,
              "v" : v,
              "access_token" : token,
              "count" : 100, 
              "thread_items_count" : 10}

Формулируем запрос, отправляем и смотрим на результат в виде словаря:

In [33]:
# есть словарь thread,
# в нем 4 ответа на комментарий – список items
# каждый ответ – один маленький словарь внутри списка 

req = requests.get(main_comm, params = params_comm)
req.json() 

{'response': {'count': 6,
  'items': [{'id': 38434,
    'from_id': 22313898,
    'date': 1678805427,
    'text': 'Высказывание харизматичное, но неверное',
    'post_id': 38433,
    'owner_id': -63442801,
    'parents_stack': [],
    'thread': {'count': 4,
     'items': [{'id': 38435,
       'from_id': 307374433,
       'date': 1678805588,
       'text': '[id22313898|Василий], а за что его односельчане уважать будут, за продвинутый Эксель?',
       'post_id': 38433,
       'owner_id': -63442801,
       'parents_stack': [38434],
       'reply_to_user': 22313898,
       'reply_to_comment': 38434},
      {'id': 38436,
       'from_id': 22313898,
       'date': 1678806132,
       'text': '[id307374433|Айгор], не путайте главу администрации с председателем совхоза (сельхозпредприятия). Главу администрации вообще мало где уважают. А председателем без уважения стать нельзя (если предприятие мелкое и все знакомые деревенские)',
       'post_id': 38433,
       'owner_id': -63442801,
       'par

### Задача 6

Давайте считать, что 100 самых новых комментариев и 10 ответов на них нам достаточно (если нет – аргумент `offset` есть, задача аналогична предыдущей по сбору постов). Повторите эту операцию выше для всех id в списке `ids` и сформируйте список с результатами. Преобразуйте результат в датафрейм `df_comm`. 

In [35]:
# по аналогии с циклом ранее, только ничего не увеличиваем
# подставляем на место post_id значение i

comments_all = []

for i in ids:
    params_comm = {"owner_id" : -63442801,
              "post_id" : i,
              "v" : v,
              "access_token" : token,
              "count" : 100, 
              "thread_items_count" : 10}
    req = requests.get(main_comm, params = params_comm)
    comm_json = req.json() 
    comm_list = comm_json["response"]["items"]
    comments_all.extend(comm_list)
    time.sleep(1)

In [36]:
# превращаем список словарей в датафрейм
# смотрим на названия столбцов
# здесь id – id поста, надо учесть для следующей задачи

df_comm = pd.DataFrame(comments_all)
df_comm.head(5)

,id,from_id,date,text,post_id,owner_id,parents_stack,thread,attachments,deleted
0,38478,252776339,1679063341,Это дискредитация помидоров!,38477.0,-63442801.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN,NaN
1,38479,221889909,1679065484,,38475.0,-63442801.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...","[{'type': 'video', 'video': {'access_key': '6b...",NaN
2,38434,22313898,1678805427,"Высказывание харизматичное, но неверное",38433.0,-63442801.0,[],"{'count': 4, 'items': [{'id': 38435, 'from_id'...",NaN,NaN
3,38440,41509235,1678894690,"Для сельского главы надо уметь в менеджмент, т...",38433.0,-63442801.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN,NaN
4,38417,691450000,1678621949,По фактам,38415.0,-63442801.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN,NaN


### Задача 7

Объедините датафрейм с собранными комментариями `df_comm` и датафрейм `with_comm10` через функцию `merge()` из `pandas`.

In [37]:
# переименовываем столбцы в with_comm10, чтобы 
# столбец с id поста назывался везде одинаково
# словарь с парами старое название: новое название
# inplace = True: сохраняем изменения сразу, 
# чтобы не писать with_comm10 = with_comm10.rename(...)

with_comm10.rename(columns = {"id" : "post_id"}, inplace = True)
with_comm10.head(2)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,post_id,date,text,nlikes,nviews,ncomments
1,38477,1679053800,Поцелуй не существует вне двух пар губ.\n\n#Де...,71,3229,1
2,38475,1678967400,"*Преподаватель перепутал сигму с эпсилон*\nНу,...",122,4680,1


In [38]:
# объединяем with_comm10 и df_comm по столбцу post_id
# left: к левому датафрейму (with_comm10) подтягиваем данные из правого (df_comm)

final = with_comm10.merge(df_comm, on = "post_id", how = "left")
final.head(5)

,post_id,date_x,text_x,nlikes,nviews,ncomments,id,from_id,date_y,text_y,owner_id,parents_stack,thread,attachments,deleted
0,38477,1679053800,Поцелуй не существует вне двух пар губ.\n\n#Де...,71,3229,1,38478.0,252776339.0,1.679063e+09,Это дискредитация помидоров!,-63442801.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN,NaN
1,38475,1678967400,"*Преподаватель перепутал сигму с эпсилон*\nНу,...",122,4680,1,38479.0,221889909.0,1.679065e+09,,-63442801.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...","[{'type': 'video', 'video': {'access_key': '6b...",NaN
2,38433,1678794600,Для сельского главы не важно уверенное владени...,138,4004,6,38434.0,22313898.0,1.678805e+09,"Высказывание харизматичное, но неверное",-63442801.0,[],"{'count': 4, 'items': [{'id': 38435, 'from_id'...",NaN,NaN
3,38433,1678794600,Для сельского главы не важно уверенное владени...,138,4004,6,38440.0,41509235.0,1.678895e+09,"Для сельского главы надо уметь в менеджмент, т...",-63442801.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN,NaN
4,38415,1678621800,"А что демократию то обсуждать, ее же нет.\n\n#...",163,4956,3,38417.0,691450000.0,1.678622e+09,По фактам,-63442801.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN,NaN


## Часть 3:  собираем информацию о пользователях

Для примера выберем пользователя с id 20473269 (пока не из датафрейма, это всего лишь я):

In [39]:
j = 20473269

Сформируем запрос:

In [40]:
# метод users.get
# все знакомые параметры
# в fields перечисляем дополнительню информацию,
# которую хотим получить
# перечисляем через запятую без пробелов

main_user = "https://api.vk.com/method/users.get"
params_user = {"access_token" : token,
               "v" : v,
               "user_id" : j,
               "fields" : "bdate,city,home_town,universities"
}
req2 = requests.get(main_user, params = params_user)
req2.json()

{'response': [{'id': 20473269,
   'bdate': '25.3.1994',
   'city': {'id': 1, 'title': 'Москва'},
   'home_town': 'Москва',
   'universities': [{'city': 1,
     'country': 1,
     'education_status': 'Студентка (бакалавр)',
     'education_status_id': 3,
     'graduation': 2016,
     'id': 128,
     'name': 'НИУ ВШЭ (ГУ-ВШЭ)'}],
   'first_name': 'Алла',
   'last_name': 'Тамбовцева',
   'can_access_closed': True,
   'is_closed': False}]}

### Задача 8

Выберите какого-нибудь пользователя, оставившего комментарий, из датафрейма и запросите по нему интересующую информацию.

In [ ]:
# up to you